# Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2023. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

## Cài đặt ban đầu

Bạn cần cài đặt môi trường để chạy được notebook này trên máy tính cá nhân của bạn. Hướng dẫn này không bao gồm phần cài đặt môi trường. Khuyến nghị: các bạn có thể cài đặt [Anaconda](https://docs.anaconda.com/free/anaconda/install/windows/).

## Cài đặt các thư viện FiftyOne và PyTorch
Hướng dẫn này dùng fiftyone là công cụ để trực quan dữ liệu và pytorch là backend chính cho các thuật toán máy học.

### Lưu ý: Đối với các bạn dùng Windows nên dùng bản fiftyone **v0.21.4**, không nên dùng bản mới nhất!

In [1]:
! pip install fiftyone==0.21.4
! pip install torch torchvision torchaudio



  Obtaining dependency information for fiftyone==0.21.4 from https://files.pythonhosted.org/packages/0e/1f/062e3087fa672a8324c969521670e5475edecb8005ab6c9f872a274db784/fiftyone-0.21.4-py3-none-any.whl.metadata
  Using cached fiftyone-0.21.4-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency information for voxel51-eta<0.11,>=0.10 from https://files.pythonhosted.org/packages/97/d1/8fb3e3c96c55a19b5498d6acc7437b12250fb092edaa9f8c16f38836897f/voxel51_eta-0.10.0-py2.py3-none-any.whl.metadata
  Using cached voxel51_eta-0.10.0-py2.py3-none-any.whl.metadata (14 kB)
Using cached fiftyone-0.21.4-py3-none-any.whl (7.7 MB)
Using cached voxel51_eta-0.10.0-py2.py3-none-any.whl (568 kB)
  Attempting uninstall: voxel51-eta
    Found existing installation: voxel51-eta 0.11.0
    Uninstalling voxel51-eta-0.11.0:
      Successfully uninstalled voxel51-eta-0.11.0
  Attempting uninstall: fiftyone
    Found existing installation: fiftyone 0.21.6
    Uninstalling fiftyone-0.21.6:
      Successfully u

Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [2]:
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz
import numpy as np
from glob import glob
import json
import os

In [ ]:
from GlobalLink import KeyframeFolder, DatasetName

Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`. 

In [3]:
name = DatasetName
dataset_dir = KeyframeFolder
dataset_type = fo.types.ImageDirectory

dataset = fo.load_dataset(name)

In [4]:
# # Tạo dataset
# name = 'aic2023-L01-L36'
# dataset_dir = 'E:/2023 HCM AI CHALLENGE/keyframes'
# dataset_type = fo.types.ImageDirectory

# dataset = fo.Dataset.from_dir(dataset_dir=dataset_dir, dataset_type=dataset_type, name=name)
# dataset.persistent = True

In [5]:
# Xem danh sách dataset
fo.list_datasets()

['2023.09.15.13.37.58', 'aic2023-L01-L20', 'aic2023-L01-L36', 'aic2023-kf-1']

In [6]:
# # Cell này dùng để xóa dataset với tên cho trước
# dataset = fo.load_dataset('aic2023-L01-L20')
# dataset.delete()

Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5151](http://localhost:5151)

In [7]:
# session = fo.launch_app(dataset, auto=False)

Hoặc bạn có thể chạy cell bên dưới để mở tab mới cho ứng dụng web fiftyone

In [8]:
# session.open_tab()

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [9]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('\\', 2)
    sample.save()

Bạn có thể xem `Sample` đầu tiên của `Dataset` bằng lệnh sau:

In [10]:
print(dataset.first())

<Sample: {
    'id': '651e209621e6bd99ec4e84a1',
    'media_type': 'image',
    'filepath': 'E:\\2023 HCM AI CHALLENGE\\keyframes\\L01_V001\\0001.jpg',
    'tags': [],
    'metadata': None,
    'video': 'L01_V001',
    'frameid': '0001',
}>


### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [11]:
# for sample in dataset:
#     object_path = f"E:\\2023 HCM AI CHALLENGE\\objects\\{sample['video']}\\{sample['frameid']}.json"
#     with open(object_path) as jsonfile:
#         det_data = json.load(jsonfile)
#     detections = []
#     for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
#         # Convert to [top-left-x, top-left-y, width, height]
#         boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
#         scoref = float(score)
        
#         # Only add objects with confidence > 0.4
#         if scoref > 0.4:
#             detections.append(
#                 fo.Detection(
#                     label=cls,
#                     bounding_box= boxf,
#                     confidence=float(score)
#                 )
#             )
#     sample["object_faster_rcnn"] = fo.Detections(detections=detections)
#     sample.save()
                           

### Thêm thông tin CLIP embedding.

In [12]:
all_keyframe = glob('E:\\2023 HCM AI CHALLENGE\\keyframes\\*\\*.jpg')
video_keyframe_dict = {}
all_video = glob('E:\\2023 HCM AI CHALLENGE\\keyframes\\*')  
all_video = [v.rsplit('\\',1)[-1] for v in all_video]

all_video[-1]

'L36_V028'

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video` 

In [13]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('\\',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [14]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [15]:
embedding_dict = {}
for v in all_video:
    clip_path = f'E:\\2023 HCM AI CHALLENGE\\clip-features-vit-b32\\{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]
    print(clip_path)

E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V001.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V002.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V003.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V004.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V005.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V006.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V007.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V008.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V009.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V010.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V011.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V012.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V013.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V014.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V015.npy
E:\2023 HCM AI CHALLENGE\clip-features-vit-b32\L01_V016.npy
E:\2023 HCM AI CHALLENGE\clip-features-v

Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [16]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)

In [17]:
already_exists = fob.similarity.Similarity.has_cached_run_results(dataset, "img_sim_32_qdrant")
already_exists

False

In [18]:
run_keys = fob.similarity.Similarity.list_runs(dataset)
run_keys

['img_sim_32_qdrant']

In [19]:
# image_index = fob.compute_similarity(
#     dataset,
#     model="clip-vit-base32-torch",      # store model's name for future use
#     embeddings=clip_embeddings,          # precomputed image embeddings    
#     brain_key="img_sim_32",
# )

In [25]:
fob.similarity.Similarity.delete_run(dataset, "img_sim_32_qdrant")

## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [21]:
# # Bạn cần phải cài version umap-learn hỗ trợ.
# fob.compute_visualization(
#     dataset, 
#     embeddings=clip_embeddings, 
#     brain_key="img_viz"
# )


In [22]:
# # Perform a text query
# query = "Sunset over city skyline, golden hues."
# view = dataset.sort_by_similarity(query, k=15, brain_key="img_sim_32")

# session.view = view

#### **Try qdrant integration**

In [26]:
qdrant_index = fob.compute_similarity(
    dataset, 
    model = "clip-vit-base32-torch",     
    embeddings=clip_embeddings,          # precomputed image embeddings  
    brain_key = "img_sim_32_qdrant", 
    backend="qdrant",
    metric="cosine",
    collection_name = "aic2023-L01-L36"
)
dataset.save()

#### **Embed text prompts**

In [ ]:
# import numpy as np
# from pkg_resources import packaging
# import torch

# device = "cuda" if torch.cuda.is_available() else "cpu"

# if packaging.version.parse(
#   torch.__version__
# ) < packaging.version.parse("1.8.0"):
#   dtype = torch.long
# else:
#   dtype = torch.int



# def get_text_embedding(prompt, clip_model):
#     tokenizer = clip_model._tokenizer

#     # standard start-of-text token
#     sot_token = tokenizer.encoder["<|startoftext|>"]

#     # standard end-of-text token
#     eot_token = tokenizer.encoder["<|endoftext|>"]

#     prompt_tokens = tokenizer.encode(prompt)
#     all_tokens = [[sot_token] + prompt_tokens + [eot_token]]

#     text_features = torch.zeros(
#         len(all_tokens),
#         clip_model.config.context_length,
#         dtype=dtype,
#         device=device,
#     )

#     # insert tokens into feature vector
#     text_features[0, : len(all_tokens[0])] = torch.tensor(all_tokens)

#     # encode text
#     embedding = clip_model._model.encode_text(text_features).to(device)

#     # convert to list for Pinecone
#     return embedding.tolist()

In [ ]:
# # 2 stages

# from fiftyone import ViewField as F

# # model = foz.load_zoo_model("clip-vit-base32-torch")
# # prompt = "Sunset over city skyline, golden hues."
# # query_vector = get_text_embedding(prompt, model)
# # stage = dataset.sort_by_similarity(query_vector[0], k=16, brain_key="img_sim_32_qdrant")

# query1 = "a laptop with some mobile phones on a table"
# query2 = "laptop AND mobile phones"
# view1 = (dataset
#         .sort_by_similarity(query1, k=25, brain_key="img_sim_32_qdrant")     
#         .sort_by_similarity(query2, k=25, brain_key="img_sim_32_qdrant")               
#         # .filter_labels("object_faster_rcnn", F("label").is_in(["Mobile phone"]))
# )

# for sample in view1:
#     print(f"{sample.video}_{sample.frameid}")

# session.view = view1.view()

L01_V009_0139
L01_V005_0130
L01_V009_0104
L01_V009_0103
L02_V005_0219
L02_V005_0218
L01_V022_0099
L02_V025_0106
L02_V005_0171
L01_V016_0120
L01_V025_0154
L01_V012_0109
L02_V025_0107
L02_V005_0187
L01_V012_0092
L02_V015_0226
L02_V001_0263
L02_V022_0192
L02_V018_0064
L02_V017_0062
L01_V022_0098
L02_V018_0325
L02_V011_0034
L02_V010_0229
L02_V024_0315


In [ ]:
# query2 = "only one man in green clothes"
# view2 = (dataset
#         .sort_by_similarity(query2, k=25, brain_key="img_sim_32_qdrant")       
# )

# for sample in view2:
#     print(f"{sample.video}_{sample.frameid}")

# session.view = view2.view()

L01_V027_0089
L02_V017_0205
L02_V001_0262
L01_V013_0110
L02_V011_0010
L02_V020_0223
L02_V020_0010
L02_V009_0235
L01_V019_0085
L02_V020_0224
L02_V008_0202
L01_V019_0086
L02_V005_0242
L02_V020_0232
L01_V019_0093
L01_V030_0159
L02_V019_0186
L01_V013_0036
L01_V019_0094
L02_V023_0249
L02_V012_0018
L02_V001_0041
L02_V001_0257
L02_V013_0256
L02_V001_0210


In [ ]:
# # 1 stage

# query = "Sunset over city skyline, golden hues."
# view = dataset.sort_by_similarity(query, k=25, brain_key="img_sim_32_qdrant")

# session.view = view.view()

In [ ]:
# fob.compute_similarity(
#     dataset, 
#     model="clip-vit-base32-torch",
#     patches_field="object_faster_rcnn",
#     brain_key = "qdrant_clip_patches", 
#     backend="qdrant",
#     metric="cosine",
#     collection_name="fiftyone-patches"
# )

In [ ]:
# model = foz.load_zoo_model("clip-vit-base32-torch")
# prompt = "a person holding a baseball bat"
# query_vector = get_text_embedding(prompt, model)

# top_k_samples = image_index.query(
#     vector=query_vector,
#     top_k=10,
#     include_values=False
# )['matches']

# # get ids of samples that most resemble a person holding a baseball bat
# top_k_ids = [res['id'] for res in top_k_samples]

# # view these samples, ordered by similarity
# view = dataset.select(top_k_ids, ordered=True)
# session.view = view.view()

AttributeError: 'SklearnSimilarityIndex' object has no attribute 'query'